In [38]:
#############
## IMPORTS ##
#############

import pandas as pd
import os.path
import numpy as np
import matplotlib
from scipy import signal


In [75]:
###################
## PREPROCESSING ##
###################
## Set the participant we're analyzing

PARTICIPANT = "user1"
DATA_1 = PARTICIPANT+'_1.csv'
DATA_2 = PARTICIPANT+'_2.csv'
DATA_3 = PARTICIPANT+'_3.csv'

#loading data in
file_paths = ['user_data\\'+PARTICIPANT+'\\'+DATA_1, 'user_data\\'+PARTICIPANT+'\\'+DATA_2, 'user_data\\'+PARTICIPANT+'\\'+DATA_1 ]
f = file_paths[0]

#accel is 10, gyro is 4
GYRO_ID = 4
ACCEL_ID = 10
gen = pd.read_csv(f, names=['Sensor', 'Time', 'X', 'Y', 'Z'], on_bad_lines='skip') #general data

#Separate data by sensor id 
gyro_df = gen[gen.Sensor == GYRO_ID]
accel_df = gen[gen.Sensor == ACCEL_ID]



#sort data by time 
gyro_df.sort_values('Time', inplace=True)
accel_df.sort_values('Time', inplace=True)

#data is now separated and ordered by time, ready for M-point filer
print(gyro_df.head())
print(accel_df.head())


    Sensor          Time        X         Y         Z
11       4  7.210914e+11 -0.11638  0.049002  0.068976
8        4  7.210947e+11 -0.11638  0.049002  0.068976
4        4  7.210969e+11 -0.11638  0.049002  0.068976
13       4  7.211012e+11 -0.11638  0.049002  0.068976
9        4  7.211076e+11 -0.11638  0.049002  0.068976
    Sensor          Time         X         Y         Z
12      10  7.210794e+11 -1.297232 -1.685075  0.404865
10      10  7.210914e+11 -1.297232 -1.685075  0.404865
2       10  7.210969e+11 -1.297232 -1.685075  0.404865
7       10  7.211076e+11 -1.297232 -1.685075  0.404865
1       10  7.211231e+11 -1.297232 -1.685075  0.404865


c:\Users\Jagrit Rai\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [81]:
##################################
# APPLYING MOVING AVERAGE FILTER #
##################################



#data is now separated and ordered by time, ready for M-point filer

#Applying Moving Average Filter 


M = 10 #M-point filter

gyro_df['MA_Time'] = gyro_df['Time'].rolling(M).mean()
gyro_df['MA_X'] = gyro_df['X'].rolling(M).mean()
gyro_df['MA_Y'] = gyro_df['Y'].rolling(M).mean()
gyro_df['MA_Z'] = gyro_df['Z'].rolling(M).mean()

accel_df['MA_Time'] = accel_df['Time'].rolling(M).mean()
accel_df['MA_X'] = accel_df['X'].rolling(M).mean()
accel_df['MA_Y'] = accel_df['Y'].rolling(M).mean()
accel_df['MA_Z'] = accel_df['Z'].rolling(M).mean()


#Creating axis vectors 

N = 30 #number of samples for a profile feature 
M_IDX = N + M #index of the Nth sample (accounts for NaNs of first M rows)

#X axis 
x_a = accel_df.loc[:, "MA_X"]
x_a = list(x_a[M-1:M_IDX]) #this is done to avoid a keyerror in the loc function 

x_g = gyro_df.loc[:, "MA_X"]
x_g = list(x_g[M-1:M_IDX])  

#y axis
y_a = accel_df.loc[:, "MA_Y"]
y_a = list(y_a[M-1:M_IDX])

y_g = gyro_df.loc[:, "MA_Y"]
y_g = list(y_g[M-1:M_IDX]) 

#z axis
z_a = accel_df.loc[:, "MA_Z"]
z_a = list(z_a[M-1:M_IDX])

z_g = accel_df.loc[:, "MA_Z"]
z_g = list(z_g[M-1:M_IDX])

features = {'x_a': x_a, 'y_a': y_a, 'z_a': z_a, 'x_g': x_g, 'y_g': y_g, 'z_g': z_g}
features_df = pd.DataFrame.from_dict(features) #mostly for presentation purposes

print(features_df)

#preprocessing is now completed

         x_a       y_a       z_a       x_g       y_g       z_g
0  -0.862802 -1.437326  0.404291 -0.102052  0.046153  0.404291
1  -0.781090 -1.364904  0.396533 -0.089642  0.040480  0.396533
2  -0.687134 -1.295530  0.390052 -0.074222  0.037284  0.390052
3  -0.594113 -1.219587  0.376317 -0.054382  0.036272  0.376317
4  -0.491355 -1.144852  0.359657 -0.034728  0.035287  0.359657
5  -0.385875 -1.022203  0.330013 -0.015793  0.033929  0.330013
6  -0.367271 -0.900867  0.273871  0.001971  0.032011  0.273871
7  -0.288467 -0.847805  0.300571  0.018163  0.029561  0.300571
8  -0.314212 -0.779240  0.289482  0.032491  0.026712  0.289482
9  -0.311314 -0.688265  0.271710  0.046792  0.023782  0.271710
10 -0.266585 -0.635453  0.292495  0.046632  0.022956  0.292495
11 -0.227954 -0.557030  0.273833  0.048390  0.024954  0.273833
12 -0.209626 -0.460679  0.254452  0.047378  0.023809  0.254452
13 -0.183360 -0.387609  0.240516  0.041678  0.019734  0.240516
14 -0.169122 -0.310146  0.229111  0.036166  0.015633  0

C:\Users\JAGRIT~1\AppData\Local\Temp/ipykernel_28716/1115689769.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gyro_df['MA_Time'] = gyro_df['Time'].rolling(M).mean()
C:\Users\JAGRIT~1\AppData\Local\Temp/ipykernel_28716/1115689769.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gyro_df['MA_X'] = gyro_df['X'].rolling(M).mean()
C:\Users\JAGRIT~1\AppData\Local\Temp/ipykernel_28716/1115689769.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

In [51]:
########################
## FEATURE EXTRACTION ##
########################



32    -0.781090
38    -0.687134
39    -0.594113
43    -0.491355
48    -0.385875
50    -0.367271
56    -0.288467
59    -0.314212
62    -0.311314
68    -0.266585
75    -0.227954
81    -0.209626
82    -0.183360
84    -0.169122
96    -0.139338
98    -0.115537
100   -0.158587
102   -0.094584
109   -0.062373
115   -0.062653
117   -0.067327
Name: MA_X, dtype: float64
